使用内置Sequential类

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)


tensor([[ 0.1652, -0.1192, -0.0615,  0.1071,  0.2496, -0.0250,  0.1717, -0.0230,
         -0.1603, -0.0846],
        [ 0.0784,  0.0620, -0.0927, -0.0066,  0.3879, -0.0105,  0.0799,  0.0489,
         -0.1952, -0.2359]], grad_fn=<AddmmBackward0>)

自定义顺序块（nn.Sequential）

In [3]:
class MySequential(nn.Module):
    def __init__(self, *args) -> None:  # 可变参数 tuple of arguments
        super().__init__()
        for block in args:  # 将每个子层拿出来 作为block
            self._modules[block] = block  # self._modules是一个有序字典 将自身作为key

    def forward(self, X):
        for block in self._modules.values():
            X = block(X)
        return X


net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
X = torch.rand(2, 20)
net(X)


tensor([[-0.1195, -0.1608, -0.0362, -0.2219, -0.1566,  0.1359,  0.2168,  0.2365,
         -0.0913,  0.0348],
        [-0.0426, -0.0823,  0.0325, -0.2559, -0.2530,  0.1185,  0.0812,  0.2659,
         -0.2026, -0.0215]], grad_fn=<AddmmBackward0>)

当你的运算多了，复杂了，nn.Sequential类便不能满足了<br>
故很多时候需要自定义类，可以在init和forward函数中做大量的自定义的灵活的运算

In [9]:
"""此例子仅用于说明构造类的灵活性，并没有什么作用"""
class FixedHiddenMLP(nn.Module):
    def __init__(self) -> None:
        super().__init__()
        self.rand_weight = torch.rand((20, 20), requires_grad=False)  # 这个weight是不参与梯度运算的，不参加训练
        self.linear = nn.Linear(20, 20)
    
    def forward(self, X):
        X = self.linear(X)
        X = F.relu(torch.mm(X, self.rand_weight) + 1)  # 矩阵乘法
        X = self.linear(X)
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)  

tensor(0.1416, grad_fn=<SumBackward0>)

In [13]:
net.forward(X)
"""
net(X) 该调用操作能使用是因为继承了来自父类Module的方法net.__call__()
父类nn.Module将该函数等价于forward()函数，故可实现前向传播，等价于net.forward(X)
"""

tensor(0.1416, grad_fn=<SumBackward0>)

还能这么用  随意搭配  足见其灵活性

In [5]:
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.4939, grad_fn=<SumBackward0>)

## 可调用对象(补充)

python中一切皆对象，函数也是对象，同时也是可调用对象（callable）。

关于可调用对象，我们平时自定义的函数、内置函数和类都属于可调用对象，但凡是可以把一对括号()应用到某个对象身上都可称之为可调用对象，

In [23]:
class Person(object):
    def __init__(self, name, gender) -> None:
        self.name = name
        self.gender = gender

p1 = Person('seek', 'male')
p1('test') # 可见报错为 对象p1不是可调用的 callable

TypeError: 'Person' object is not callable

一个类实例要变成一个可调用对象，只需要实现一个特殊方法__call__()

In [24]:
class Person(object):
    def __init__(self, name, gender) -> None:
        self.name = name
        self.gender = gender
        
    def __call__(self, friend):
        print(f'My name is {self.name}')
        print(f'My friend is {friend}')

p2 = Person('seek', 'male')
p2('Li Hua') # 此时变为callable

My name is seek
My friend is Li Hua


In [25]:
callable(p1), callable(p2)

(False, True)